In [1]:
import pandas as pd
from tbdynamics.tools.plotting import plot_model_vs_actual
import nevergrad as ng
from tbdynamics.settings import CM_PATH
# Import our convenience wrapper
from estival.wrappers.nevergrad import optimize_model
from tbdynamics.camau.calibration.utils import get_bcm, load_targets
from multiprocessing import cpu_count

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
pd.options.plotting.backend = "plotly"

In [ ]:
params = {
    "start_population_size": 30000.0,
    "seed_time": 1805.0,
    "seed_num": 1.0,
    "seed_duration": 1.0,
    # "contact_rate": 0.02,
    "rr_infection_latent": 0.1890473700762809,
    "rr_infection_recovered": 0.17781844797545143,
    "smear_positive_death_rate": 0.389,
    "smear_negative_death_rate": 0.025,
    "smear_positive_self_recovery": 0.231,
    "smear_negative_self_recovery": 0.130,
    "screening_scaleup_shape": 0.5,
    "screening_inflection_time": 1993,
    # "time_to_screening_end_asymp": 2.1163556520843936,
    "acf_sensitivity": 0.90,
    # "prop_mixing_same_stratum": 0.6920672992582717,
    # "early_prop_adjuster": -0.017924441638418186,
    # "late_reactivation_adjuster": 1.1083422207175728,
    "detection_reduction": 0.3,
    # "total_population_dispersion": 3644.236227852164,
    # "notif_dispersion": 88.37092488550051,
    # "latent_dispersion": 7.470896188551709,
}
covid_effects = {"detection_reduction": True, "contact_reduction": False}
bcm = get_bcm(params, covid_effects)

In [4]:
# Set the number of workers for parallel optimization
orunner = optimize_model(bcm, opt_class=ng.optimizers.TwoPointsDE, num_workers=cpu_count())
for i in range(8):
    rec = orunner.minimize(500)
mle_params = rec.value[1]
mle_params

{'contact_rate': 0.02520477457868967,
 'rr_infection_latent': 0.2126101133062807,
 'rr_infection_recovered': 0.13561538227034747,
 'smear_positive_death_rate': 0.40520978718786593,
 'smear_negative_death_rate': 0.02817314911378398,
 'smear_positive_self_recovery': 0.2361729402208079,
 'smear_negative_self_recovery': 0.1824688631421079,
 'time_to_screening_end_asymp': 1.972001796338878,
 'early_prop_adjuster': -0.07336348976490145,
 'late_reactivation_adjuster': 1.1064772281255661,
 'detection_reduction': 0.291969003921787,
 'total_population_dispersion': 4876.9400804787365,
 'notif_dispersion': 79.16110233619047,
 'latent_dispersion': 11.935571656374927}

In [5]:
res = bcm.run(mle_params)
derived_df_0 = res.derived_outputs
targets = load_targets(CM_PATH / 'targets.yml')

In [6]:
model = bcm.model

In [7]:
model.get_default_parameters()

In [8]:
plot_model_vs_actual(
    derived_df_0, targets['total_population'], "total_population", "Population", "Modelled vs Data"
)

In [9]:
plot_model_vs_actual(
    derived_df_0, targets['notification'], "notification", "Notifications", "Modelled vs Data"
)

In [10]:
derived_df_0['percentage_latent'].plot()

In [11]:
derived_df_0['detection_rate'].plot()

In [12]:
derived_df_0['percentage_latent_adults'].plot()

In [13]:
derived_df_0['prevalence_infectious'].plot()

In [14]:
derived_df_0.loc[2014:2019,[f'total_populationXact3_{act3_stratum}' for act3_stratum in ["trial","control"]]].plot()

In [15]:
derived_df_0.loc[2014:2019,'acf_detectionXact3_trialXorgan_pulmonary'].plot()

In [16]:
derived_df_0.loc[2016:2019,'acf_detectionXact3_controlXorgan_pulmonary'].plot()